In [1]:
import numpy as np
from pathlib import Path
import time
import matplotlib.pyplot as plt
start_time = time.time()

import xobjects as xo
import xtrack as xt
import xpart as xp
import xobjects as xo
import xcoll as xc

import apertls
from matplotlib.colors import LogNorm

In [2]:
# This script takes around 8 minutes on a modern CPU (80s preparation+interpolation, 400s tracking)
beam = 1
plane = 'DNeg'#'DPpos'

num_particles  = 5000
sweep          = 3500
sweep          = -abs(sweep) if plane == 'DPpos' else abs(sweep)
#num_turns      = int(20*abs(sweep))
num_turns=6000

In [3]:
nemitt_x = 3.5e-6
nemitt_y = 3.5e-6

In [4]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta.json')

#setting all cavities to 200 (even 800 cavities) because rf sweep not implemented
cavity_elements, cavity_names = line.get_elements_of_type(xt.Cavity)
for name in cavity_names:
    line[name].frequency = 200e6
    line[name].lag = 180

line['acl.31735'].voltage = 0


Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


In [5]:
line.get_s_position(['bpcn.12508.a_aper','bpcn.12508.b_aper'])

[np.float64(799.5902), np.float64(799.825)]

In [6]:
799.825 - 799.5902

0.23480000000006385

In [7]:
theta = np.atan(5.3/809)
L1 = 34e-3
L2 = 21e-3
L3 = 754e-3
h_tot = 5.3e-3

x1 = L1/np.cos(theta)
x2 = L2/np.cos(theta)
x3 = L3/np.cos(theta)

print(np.sin(theta)*(x1+x2+x3), h_tot)

x_tot = x1+x2+x3
w_steel = x1/x_tot
w_air = (x2+x3)/x_tot

print(w_steel, w_air)

0.0053 0.0053
0.04202719406674908 0.9579728059332508


In [8]:
d_steel_air = x2+x3

X_Be = 35.28e-2
X_air = 303.9
X_Fe = 1.757e-2
X_C = 19.32e-2
X_steel = (0.02/X_C + 0.98/X_Fe)**(-1)
X_steel_air = (w_steel/X_steel + w_air/X_air)**(-1)

d_Be = X_Be/X_steel_air * d_steel_air #shorter than d_steel_air so we can keep angle theta and stop collimator before end of VCAK

In [9]:
import pdb

In [10]:
%debug
line.discard_tracker()
#collimator jaw positions:
ap_vebb_x = 0.0645
jaw_ru = -ap_vebb_x
jaw_rd = -ap_vebb_x - h_tot
jaw_lu = ap_vebb_x
jaw_ld = ap_vebb_x - h_tot
L_coll = d_Be

coll_12501 = xc.EverestCollimator(length=L_coll, material=xc.materials.Beryllium, jaw_RU=jaw_ru, jaw_RD=jaw_rd, jaw_LU=jaw_lu, jaw_LD=jaw_ld)
coll_61101 = xc.EverestCollimator(length=L_coll, material=xc.materials.Beryllium, jaw_RU=jaw_ru, jaw_RD=jaw_rd, jaw_LU=jaw_lu, jaw_LD=jaw_ld)
name_colls = ['vcak.12501_coll', 'vcak.61101_coll']
s_colls = [line.get_s_position('vebb.12490.b_aper'), line.get_s_position('vebb.61090.b_aper')]

names_vcak_apers_to_remove = ['vcak.12501.a_aper', 'vcak.12501.b_aper', 'vcak.61101.a_aper', 'vcak.61101.b_aper']
# for name in names_vcak_apers_to_remove:
#     line.remove(name)

line.collimators.install(names=name_colls, elements=[coll_12501, coll_61101], at_s=s_colls)
# line.collimators.install(names=name_colls[0], elements=[coll_12501], at_s=s_colls[0])
# line.insert(name_colls[0], coll_12501, at=s_colls[0])

ERROR:root:No traceback has been produced, nothing to debug.


Slicing line:   0%|          | 0/75280 [00:00<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'length'

In [ ]:
line.get_s_position('vcak.12501.a_aper')

np.float64(797.7962)

In [ ]:
tt = line.get_table()

In [ ]:
tt.rows['vcak.61101.*']

Table: 2 rows, 11 cols
name                          s element_type isthick isreplica parent_name iscollective ...
vcak.61101.a_aper       6109.41 Drift           True     False None               False
vcak.61101.b_aper       6110.27 LimitEllipse   False     False None               False

In [ ]:
s_colls

[np.float64(797.7953), np.float64(6109.4139)]

In [ ]:
line.get_length()

np.float64(6911.50379999983)

In [ ]:
idx = line.element_names.index('vebb.12490.b_aper')
line.element_names[idx-2:idx+10]

['mbb.12490..10_aper2',
 'drift_218..0',
 'vebb.12490.b_aper',
 'drift_218..1',
 'drift_218..2',
 'drift_218..3',
 'vtto.12505.a_aper',
 'drift_218..4',
 'lsd.12505',
 'drift_219..0',
 'vtto.12505.b_aper',
 'drift_219..1']

In [21]:
idx = line.element_names.index('drift_218..0')
line.element_names[idx:idx+10]

['drift_218..0',
 'vebb.12490.b_aper',
 'drift_218..1',
 'vcak.12501.a_aper',
 'drift_218..2',
 'vcak.12501.b_aper',
 'drift_218..3',
 'vtto.12505.a_aper',
 'drift_218..4',
 'lsd.12505']

In [11]:
line['drift.218..1']

KeyError: 'Name drift.218..1 not found'

In [14]:
tt.rows['drift.218.*']

Table: 5 rows, 11 cols
name                     s element_type isthick isreplica parent_name iscollective       s_start ...
drift_218..0         797.6 DriftSlice      True     False drift_218          False         797.6
drift_218..1       797.795 DriftSlice      True     False drift_218          False       797.795
drift_218..2       797.796 DriftSlice      True     False drift_218          False       797.796
drift_218..3       798.655 DriftSlice      True     False drift_218          False       798.655
drift_218..4       798.656 DriftSlice      True     False drift_218          False       798.656

In [13]:
tt = line.get_table()

In [18]:
line['drift_218..0']

View of DriftSlice(parent_name='drift_218', _parent=None, radiation_flag=np.int64(10), delta_taper=0, weight=0.172)